# Importation des bibliothéques

In [1]:
!pip install graphframes

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip install --upgrade pyarrow

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import pandas as pd
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql.functions import col
from pyspark.sql import functions as F

from igraph import Graph
from pyspark.sql.window import Window
from pyspark.sql.functions import col

from functools import reduce
from pyspark.sql.functions import collect_set, min as min_, expr,array_min,max
from pyspark.sql.functions import lit

from graphframes import GraphFrame

In [4]:
!find /root -name "pyspark" |grep bin

/root/anaconda3/bin/pyspark
/root/anaconda3/pkgs/pyspark-3.1.2-pyhd3eb1b0_0/site-packages/pyspark/bin/pyspark
/root/anaconda3/lib/python3.9/site-packages/pyspark/bin/pyspark


In [5]:
!which java

/usr/bin/java


In [6]:
!java -version

openjdk version "11.0.14" 2022-01-18
OpenJDK Runtime Environment (build 11.0.14+9-post-Debian-1deb10u1)
OpenJDK 64-Bit Server VM (build 11.0.14+9-post-Debian-1deb10u1, mixed mode, sharing)


In [7]:
import os
os.environ["SPARK_HOME"] =  "/root/anaconda3/lib/python3.9/site-packages/pyspark" 
os.environ["JAVA_HOME"] ="/usr"

# Lancer Spark

In [18]:
import findspark 
print("findspark.init() initialise les variables d'environnement pour spark") 
findspark.init() 

from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# import pyspark functions
import pyspark.sql.functions as f

from pyspark.sql.types import StringType, IntegerType, LongType, FloatType, ArrayType,StructType,StructField, BooleanType


findspark.init() initialise les variables d'environnement pour spark


In [19]:
def demarrer_spark():
    
  # 60 cores 
  NB_CORES = 5
  # NB_CORES = 64
  local = f"local[{NB_CORES}]"
  # local = "local[*]"
    

  # le parametre spark.local.dir indique le repertoire contenant les données temporaires ecrites sur disque lorsque le shufle ne tient pas en memoire
  
    
  appName = "PLDAC"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "320G").\
  set("spark.driver.memory","320G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.driver.maxResultSize", "20G").\
  set("spark.local.dir", "/data/bd/spark/tmp").\
  set("spark.sql.execution.arrow.pyspark.enabled", "true")
    

# # to allow sharing in memory arow format between pandas and spark : speeds up the creation of a spark df from a pandas df
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
    
  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (NB_CORES coeurs)
  shuffle_partitions = 3 * NB_CORES
  print("shuffle", shuffle_partitions)
  spark.conf.set("spark.sql.shuffle.partitions", str(shuffle_partitions))    

  print("session démarrée, son id est ", sc.applicationId)
  return spark

spark = demarrer_spark()

shuffle 15
session démarrée, son id est  local-1712144199484


# Calcul en parallèle des composantes connectées pour tous les fichiers de partition

In [10]:
def calculate_partition_components(partition_folder):
    def components(partition_file) :
        df = pd.read_parquet(partition_file)
        g = Graph.TupleList(df[['query_id', 'target_id']].itertuples(index=False), directed=False)
        connected_components = g.connected_components()
        node_ids=[]
        for component in connected_components:
            node_ids.append([g.vs[node_index]['name'] for node_index in component])
        #on trie en fonction de la taille 
        composantes_triees = sorted(node_ids, key=len, reverse=True)
        # Créer une liste de lignes pour chaque composante
        rows = []
        for i, component in enumerate(composantes_triees):
            for seqid in component:
                rows.append((seqid , i))
        return rows

    # Schéma du DataFrame
    schema = StructType([
        StructField("file", StringType(), False)
    ])
    
    # Données à partir desquelles créer le DataFrame
    #data = [(file,) for file in os.listdir(partition_folder)]
    data = [(i,) for i, file in enumerate(os.listdir(partition_folder))]

    # Création du DataFrame
    df = spark.createDataFrame(data, schema).repartition(10)

    # Enregistrer la fonction UDF
    schema_components = StructType([
        StructField("seqID", StringType(), nullable=False),
        StructField("component_id", IntegerType(), nullable=False)
    ])

    components_udf = F.udf(components, ArrayType(schema_components))

    #df_with_components = df.withColumn("components", components_udf(F.concat(F.lit(partition_folder+"/"), df["file"])))
    df_with_components = df.withColumn(
        "components",
        components_udf(
            F.concat(
                F.lit(partition_folder + "/partition_"), 
                df["file"] , 
                F.lit(".parquet")
            )
        )
    )
    
    return df_with_components

# Calcul des composantes connexes du graph complet

In [11]:
def components_2(edges):
    # Créer un graphe à partir des arêtes
    g = Graph.TupleList(edges, directed=False)
    
    connected_components = g.connected_components()
    node_ids=[]
    for component in connected_components:
        node_ids.append([g.vs[node_index]['name'] for node_index in component])
    #on trie en fonction de la taille 
    composantes_triees = sorted(node_ids, key=len, reverse=True)
    # Créer une liste de lignes pour chaque composante
    rows = []
    for i, component in enumerate(composantes_triees):
        for seqid in component:
            rows.append(Row(partial_component=seqid ,component=i ))
    
    # Créer un DataFrame Spark à partir de la liste de lignes
    result_df = spark.createDataFrame(rows)
    return result_df

In [12]:
def calculate_components(folder_partition ):
    # Claclul des composantes partielles dans chaque partition
    df_with_components = calculate_partition_components(folder_partition)
    df_exploded = df_with_components.select('file',  explode('components').alias('component'))
    df_split = df_exploded.select("file", col("component.seqID").alias("seqID"), col("component.component_id").alias("component_id"))
    
    # Ajouter une colonne d'identifiant unique basée sur le couple (file, component_id)
    windowSpec = Window.orderBy("file", "component_id")
    df = df_split.withColumn("new_component_id", dense_rank().over(windowSpec))
    new_df = df.select("seqID","new_component_id")

    # new_df = df_split.select(
    #             'seqID',
    #             concat(col('file'), lit(':'), col('component_id')).alias('new_component_id')
    #         )
    
    #cration d'un graph ou chaque noeud est le numero d'une composante (new_graph)

    components_per_node = new_df \
        .groupby("seqID") \
        .agg(collect_set("new_component_id").alias("components")) \
        .withColumn("min_component", array_min("components"))

    # components_per_node = new_df \
    #     .groupby("seqID") \
    #     .agg(collect_set("new_component_id").alias("components")) \
    #     .withColumn("min_component", array_min("components"))


    components_per_node_exploded = components_per_node.select('seqID', 'min_component', explode('components').alias('component'))
    result = components_per_node_exploded.where(col("component") > col("min_component")).drop("seqID").dropDuplicates()
  
    
    new_graph = result.withColumnRenamed("min_component", "src").withColumnRenamed("component", "dst")

    #calcul des composantes sur le nouveau graph
    edges =new_graph.select('src', 'dst').collect()
    composantes_df = components_2(edges)

    composantes_df_exploded = composantes_df \
            .groupby("component") \
            .agg(collect_set("partial_component").alias("partial_components")) \
            .withColumn("min_partial_component", array_min("partial_components")) \
            .selectExpr("explode(partial_components) as partial_component" , "min_partial_component")
    
    #resultat final en faisant une jointure gauche
    result = components_per_node.join(composantes_df_exploded, components_per_node.min_component == composantes_df_exploded.partial_component, "left_outer") \
               .select(components_per_node.seqID, when(composantes_df_exploded.partial_component.isNull(), components_per_node.min_component).otherwise(composantes_df_exploded.min_partial_component).alias("new_component_id"))


    #ecriture du résultat dans des fichier parquet
    file_path = f"/data/bd/dataset/proteine/80_80/G99/graph_nodeID_only"
    result.repartition(1).write.mode("overwrite").parquet(file_path)
    
    return result

# Calcul des composantes avec la nouvelle méthode

In [17]:
import time

folder_partition = "/data/bd/dataset/proteine/80_80/G99/graph_nodeID_only_partitionned_200/"

start_time = time.time()
result_df = calculate_components(folder_partition)
end_time = time.time()
print("Temps d'execution avec la nouvelle methode : ", end_time - start_time)

ERROR:root:KeyboardInterrupt while sending command.                (0 + 9) / 10]
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/root/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [15]:
result_df = calculate_partition_components(folder_partition)


In [16]:
result_df.show()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/root/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 